                                                                                       4camping_workspace
 1 - pojmenovano tabulky ve storeg (CAPS_LOCK)
- tabulky z DB nahrany pres STORAGE
- tabulky ohledne county, region nahrany pres COMONENT/GOOGLE DRIVE
 
info: 
okres - county
kraj - region
tabulk  
- COUNTY_CZ : 
- ZIP_CODE_CITY: 
- ZIP_CODE_STORES: 

- vice o vztazich v datovém modelu
https://lucid.app/lucidchart/5b471e84-a440-4587-a1cf-19c93dbd04c6/edit?invitationId=inv_0c0c0ddc-16c1-424f-ae3e-6f41d12bd1f1&page=wbRCCwA8fE17#

 
 

#### 1_4CAMPING_ORDERS_PRODUCT - Privni transformace 
- VNORENY SELECT UNION PRODUKTU, JOINOVANI S CAT, PROD, ORDERS, SHIPPINGMETHODS
- spojit ordes_product nasledne spojime jeste do toho tabulky orders, products, shipping methods a categoris
- Vystupní tabulka bude : ORDER_PRODUCTS_0

In [ ]:
CREATE OR REPLACE TABLE ORDER_PRODUCTS_0 AS ( 
--# select na spojenné tabulky, orders_product, orders, product, chategories, shippinh methods
SELECT 
    OP0.*
    ,O."city" AS CITY
    ,O."date" AS DATES
    ,O."shipping_methods_id" 
    ,O."number" AS ORDER_NUMBER --CISLO OBJEDNÁVKY
    ,P."name_cs" AS PRODUCT_NAME
    ,C."name_cs" AS SUBCATEGORIES_NAME
    ,SM."name_cs" AS SHIPPING_METHOD
    ,SPLIT(C."name_full_cs", '||')[0] AS CATEGORIES --##vytvorime categories pomoci splitem z sloupcu z tabulky category
   
FROM 
--## 3 orders_product tabulky spojime dohromady
(SELECT
   OP1."id" AS ID
   ,OP1."date_inserted"
   ,OP1."orders_id" AS ORDERS_ID
   ,OP1."products_id" AS PRODUCTS_ID
   ,OP1."variant_name_cs" AS VARIANT_NAME_CS
   ,OP1."quantity" AS QUANTITY
   ,OP1."price_czk" AS PRICE_CZK
   ,OP1."weight" AS WEIGHT
FROM ORDER_PRODUCT_1 AS OP1

UNION ALL

SELECT
    OP2."id"
    ,OP2."date_inserted"
   ,OP2."orders_id"
   ,OP2."products_id"
   ,OP2."variant_name_cs"
   ,OP2."quantity"
   ,OP2."price_czk"
   ,OP2."weight"
FROM ORDER_PRODUCT_2 as OP2

UNION ALL

SELECT
    OP3."id"
    ,OP3."date_inserted"
   ,OP3."orders_id"
   ,OP3."products_id"
   ,OP3."variant_name_cs"
   ,OP3."quantity"
   ,OP3."price_czk"
   ,OP3."weight"
FROM ORDER_PRODUCT_3 as OP3)  AS OP0

INNER JOIN ORDERS AS O --INNER JOIN JE PROTO, ZE TABULKY ORDERS A PRODUCT_ORDERS BYLY STAZENY V RUZNOU DOBU A NE VSECHNY PO JSOU V O
ON OP0.ORDERS_ID = O."id"
LEFT JOIN PRODUCTS AS P
ON OP0.PRODUCTS_ID = P."id"
LEFT JOIN CATEGORIES AS C
ON C."id" = P."categories_id"
LEFT JOIN SHIPPING_METHODS AS SM
ON SM."id" = O."shipping_methods_id"
)

#### 1_4CAMPING_ORDERS_PRODUCT - Druhá transformace
- PRIDAVANI POTREBNYCH SLOUPCU DO TABULKY ORDER_PRODUCTS0 (DATE, ZIP, COUNTY, REGION, COLOR)

PRACE SE SLOUPCE DATE - VYTVORENI NOVYCH SLOUPCU

In [ ]:
--upravim tabulku ORDER_PRODUCTS_0 - pridam  nove sloupce: pro praci s datumem
ALTER TABLE ORDER_PRODUCTS_0
ADD COLUMN DAY_NAME VARCHAR(20), DAY_NAME_CS VARCHAR(20), IDAY INTEGER, IYEAR INTEGER, IMONTH INTEGER, IDATE DATE;


--do upravene tabulky orders naplnim prislusne sloupecky
UPDATE ORDER_PRODUCTS_0
SET DAY_NAME = dayname(DATES),
    DAY_NAME_CS =
    CASE
        WHEN DAY_NAME = 'Sun' THEN 'Neděle'
        WHEN DAY_NAME = 'Mon' THEN 'Pondělí'
        WHEN DAY_NAME = 'Tue' THEN 'Úterý'
        WHEN DAY_NAME = 'Wed' THEN 'Středa'
        WHEN DAY_NAME = 'Thu' THEN 'Čtvrtek'
        WHEN DAY_NAME = 'Fri' THEN 'Pátek'
        WHEN DAY_NAME = 'Sat' THEN 'Sobota'
            END,
    IDATE = DATE(TO_TIMESTAMP(DATES, 'YYYY-MM-DD HH24:MI:SS')),
    IYEAR = DATE_PART('YEAR', IDATE),
    IMONTH = DATE_PART('MONTH', IDATE),
    IDAY = DATE_PART('DAY', IDATE);

#### OBOHACENI TAB ORDER_PRODUCT_0 - BARVY, VELIKOST (!!!! DODELAT), VALUE, UPRAVA FORMATU

- MAME PROBLEM PRAZDNYCH HODNOT VE SLOUPCICH, SE KTERYMI CHCEME POCITAT - ZJISTILY JSME, ZE SE JEDNA O CHYBEJICI HODNOTU U PRODUKTU SLUNECNI BRYLE. DOPOCITALY JSME PRUMERNOU CENU A 
223 czk


In [ ]:
SELECT PRODUCTS_ID, AVG(PRICE_CZK) AS PRUMER
FROM ORDER_PRODUCTS_0
WHERE PRODUCTS_ID = '18409' AND PRICE_CZK <> ''
GROUP BY PRODUCTS_ID

In [ ]:
--### OLD VERSION
--upravim tabulku ORDER_PRODUCT_CD - pridam sloupecek DAY_NAME
ALTER TABLE ORDER_PRODUCT_CD
ADD COLUMN DAY_NAME VARCHAR(20);

--upravim tabulku ORDER_PRODUCT_CD - pridam sloupecek DAY_NAME_CS
ALTER TABLE ORDER_PRODUCT_CD
ADD COLUMN DAY_NAME_CS VARCHAR(20);

--upravim tabulku ORDER_PRODUCT_CD - pridam sloupecek IDAY
ALTER TABLE ORDER_PRODUCT_CD 
ADD COLUMN IDAY INTEGER;

--upravim tabulku ORDER_PRODUCT_CD - pridam sloupecek IYEAR
ALTER TABLE ORDER_PRODUCT_CD 
ADD COLUMN IYEAR INTEGER;

--upravim tabulku ORDER_PRODUCT_CD - pridam sloupecek IMONTH
ALTER TABLE ORDER_PRODUCT_CD 
ADD COLUMN IMONTH INTEGER;

--upravim tabulku ORDER_PRODUCT_CD - pridam sloupecek IMONTH
ALTER TABLE ORDER_PRODUCT_CD 
ADD COLUMN IDATE DATE;

--do upravene tabulky orders naplnim sloupecek day_name dle sloupecku "date"
UPDATE ORDER_PRODUCT_CD
SET DAY_NAME = dayname("date");

--do upravene tabulky orders naplnim sloupecek day_name_cs dny v tydnu v cestine
UPDATE ORDER_PRODUCT_CD
SET DAY_NAME_CS =
    CASE
        WHEN DAY_NAME = 'Sun' THEN 'Neděle'
        WHEN DAY_NAME = 'Mon' THEN 'Pondělí'
        WHEN DAY_NAME = 'Tue' THEN 'Úterý'
        WHEN DAY_NAME = 'Wed' THEN 'Středa'
        WHEN DAY_NAME = 'Thu' THEN 'Čtvrtek'
        WHEN DAY_NAME = 'Fri' THEN 'Pátek'
        WHEN DAY_NAME = 'Sat' THEN 'Sobota'
            END;

--PREDELAM VARCHAR DATUM S TIMESTAMP NA TIMESTAMP, A PAK NA DATE YYYY-MM-DD
UPDATE ORDER_PRODUCT_CD
SET IDATE = DATE(TO_TIMESTAMP("date", 'YYYY-MM-DD HH24:MI:SS'));

--PRAZDNE SLOUPCE IYEAR, IMONTH A IDAY NAPLNIM PRISLUSNOU INFORMACI ZE SLOUPECKU TIMESTAMP_DATE
UPDATE ORDER_PRODUCT_CD
SET 
    IYEAR = DATE_PART('YEAR', IDATE),
    IMONTH = DATE_PART('MONTH', IDATE),
    IDAY = DATE_PART('DAY', IDATE);

potom jsem joinovala tabulkou co udelala Anna, a pripjoila jsem tabilky:
orders --> pokud v tabulce nehame cislo objednavky (number), a je nejaky nejastnost rychle muzu kontrolovat
products --> je potreba kvuli kategoriam
categories
shipping methods

In [ ]:
SELECT
 opl."id" orders_product_id
,SPLIT(c."name_full_cs", '||')[0] AS nadkategory  --# ze slupce fullname, se da s tim udelat nadkategorie
,cd."date" 
,cd.day_name
,cd.day_name_cs
,cd.iday
,cd.iyear
,cd.imonth
,cd.idate
,o."number" cislo_objednavky
,opl."orders_id" orders_id
,opl."products_id" products_id
,opl."variant_name_cs" product_variant_name
,opl."quantity" pocet_produktu
,opl."price_czk"
,opl."weight"
,p."name_cs" product_name
,c."name_cs" categories_name
,cd."city"
,sm."name_cs" shipping_method
FROM orders_product_l AS opl
LEFT JOIN "products" AS p ON opl."products_id" = p."id"
LEFT JOIN "categories" AS c ON  c."id" = p."categories_id"
LEFT JOIN order_product_cd AS cd on cd.id_order_product = opl."id"
LEFT JOIN "orders" AS o ON o."id" = opl."orders_id"
LEFT JOIN "shipping_metods" AS sm ON sm."id" = o."shipping_methods_id"

A pak ja jsem k tomu nejak pridala dopravce , kod je ok, ale musime pak zakomponovat lip

In [ ]:
CREATE OR REPLACE TABLE FORCAMPING_ORDERS_2 AS
SELECT *, 
  CASE 
    WHEN shipping_method LIKE '%GLS%' THEN 'GLS' 
    WHEN shipping_method  LIKE '%DPD%' THEN 'DPD' 
    WHEN shipping_method  LIKE '%GEIS%' THEN 'GEIS' 
    WHEN shipping_method  LIKE '%PPL%' THEN 'PPL' 
    WHEN shipping_method  LIKE '%Balíkovna%' THEN 'BALIKOVNA' 
    WHEN shipping_method  LIKE '%iftago%' THEN 'LIFTAGO' 
    WHEN shipping_method  LIKE '%Česká%' THEN 'ČESKÁ POŠTA' 
    WHEN shipping_method  LIKE '%loženka%' THEN 'ULOŽENKA' 
    WHEN shipping_method  LIKE '%Dodo%' THEN 'DODO' 
    WHEN shipping_method  LIKE '%Packeta%' THEN 'PACKETA' 
    WHEN shipping_method  LIKE '%silkovna%' THEN 'PACKETA' 
    WHEN shipping_method  LIKE '%DHL%' THEN 'DHL'
    WHEN shipping_method  LIKE '%SPS%' THEN 'SPS - SK'
    WHEN shipping_method  LIKE '%venská po%' THEN 'SK pošta'
    WHEN shipping_method  LIKE '%test%' THEN 'test'
    ELSE 'OSOBNÍ ODBĚR' 
  END AS dopravce
FROM  FORCAMPING_ORDERS_2

Tady nejak jsem snazila odstranit ten psatny datum, ktreý nam zlobil

In [ ]:
CREATE OR REPLACE TABLE FORCAMPING_ORDERS_FINAL_ALL AS

SELECT 
 orders_product_id
,cislo_objednavky
,orders_id
,products_id
,product_variant_name
,pocet_produktu
,"price_czk"
,"weight"
,product_name
,categories_name
,shipping_method
,day_name
,day_name_cs
,iday
,iyear
,imonth
,idate
,nadkategory
,city_from_shipping
,city_and_cfs
,DOPRAVCE
FROM FORCAMPING_ORDERS_FINAL

pridani balikobot + psc okres z CP pouze 2022

In [ ]:
CREATE OR REPLACE TABLE FORCAMPING_ORDERS_2022_KOMPLET AS
SELECT
    fofa.*
    ,"rec_city" AS REC_CITY
    ,"rec_zip" AS REC_ZIP
    ,po.PSC
    ,po.NAZOKRESU
    ,PO.NAZCOBCE
    ,ok."Nazev_kraje"
FROM FORCAMPING_ORDERS_FINAL_ALL  AS fofa -- fofa je tabulka vsech orders
LEFT JOIN "fc_balikobot" AS f  --FC_BALIKOBOT NEMA ORDERS_ID, KTERE BYLY OSOBNI ODBER NA PRODEJNE
ON f."order_id" = fofa.orders_id
LEFT JOIN "psc_okres" as po
ON po.PSC = f."rec_zip"
LEFT JOIN "okres_kraj" as ok
ON OK."Nazev_okresu" = po.NAZOKRESU
WHERE "rec_country" = 'CZ' 
AND IYEAR = '2022'

A pak jeste mame regex

In [ ]:
SELECT TRIM(REPLACE(regexp_substr(PRODUCT_VARIANT_NAME, 'Barva:([^\/]*)'),'Barva: ', '')),
PRODUCT_VARIANT_NAME,
*
FROM FORCAMPING_ORDERS_FINAL_ALL;
LIMIT 100;

přetipovani sloupcu na INT a vynasobeni pocet_produktu s cenama
(porodany dany polozek v objednavce * cena za jednoho polozku)

In [ ]:
SELECT 
 CAST (pocet_produktu AS INTEGER) as pocet_produktu_int
, CAST ("price_czk" AS INTEGER) AS price_per_item
, pocet_produktu_int * price_per_item as CELKOVA_CENA
FROM FORCAMPING_ORDERS_FINAL_ALL
Limit 10